In [ ]:
import torch
import torch.nn as nn
from pop import Pilot, Camera
import time
from torchvision import models, transforms
from PIL import Image
import numpy as np
import cv2

camera = Camera(300, 300)
dc = Pilot.Data_Collector("Track_Follow", camera)

# 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ MobileNetV2 구조 정의 및 classifier 교체
model = models.mobilenet_v2(pretrained=False)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)

# ✅ 모델 로드
try:
    model.load_state_dict(torch.load("best_model_v2.pth", map_location=device))
    print("✅ 모델 로딩 성공")
except Exception as e:
    print(f"❌ 모델 로딩 실패: {e}")
    exit(1)

model.to(device)
model.eval()

# ✅ MobileNetV2용 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# ✅ 자동차와 카메라 초기화
autocar = Pilot.AutoCar()
camera = Camera(300,300)

# ✅ 손바닥 인식 함수 (0: blocked, 1: free)
def detect_hand(frame):
    if isinstance(frame, np.ndarray):
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pil_image = Image.fromarray(frame)
    input_tensor = transform(pil_image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(input_tensor)
    _, predicted = torch.max(output, 1)
    return predicted.item() == 0

try:
    while True:
        frame = camera.value

        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("🛑 Q 누르어서 종료")
            break

        if detect_hand(frame):
            print("패드 감지됨! 정지!")
            autocar.stop()
        else:
            print("패드 감지 안됨, 8자 움직임 시작")
            autocar.steering = -1
            autocar.forward(50)
            time.sleep(3)
            autocar.steering = 1
            autocar.forward(50)
            time.sleep(3)

        time.sleep(0.5)

except KeyboardInterrupt:
    print("🛑 Ctrl+C로 종료됨")
finally:
    autocar.stop()
    # camera.release()  # 이 라이브러리에서는 제거해야 함
    cv2.destroyAllWindows()
    print("프로그램 종료 완료")
